In [4]:
import openai
from openai.error import ServiceUnavailableError, Timeout
import json
from json import JSONDecodeError
import pathlib
from tqdm import tqdm
from time import sleep
from data.openai_api_key import key

path_to_dataset = pathlib.Path('./dataset')

openai.api_key = key

# prompt = (f"Jesteś światowej sławy generatorem bajek dla dzieci o dowolnym temacie i treści o długości około 3000 słów. "
#           f"Twoja odpowiedź powinna być w formacie JSON: {'title': 'tytuł bajki', 'content': 'treść bajki'}. "
#           f"Twoja odpowiedź nie powinna zawierać żadnej innej treści oprócz JSON-a")

prompt = f"Jesteś światowej sławy generatorem bajek dla dzieci w języku polskim o dowolnym temacie i treści o długości nie większej niż 2500 słów. Twoja odpowiedź nie powinna zawierać żadnej innej treści oprócz treści bajki."

conversation_history = [
    {"role": "system", "content": prompt},
]

In [ ]:
samples_to_generate = 200
temperature = 1.0

for _ in tqdm(range(samples_to_generate)):
    try:
        response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=conversation_history,
        timeout=60,
        temperature=temperature
        )
    except:
        sleep(60)
        continue
    
    response_metadata = {
        "id": response['id'],
        "object": response['object'],
        "created": response['created'],
        "model": response['model'],
        "usage": response['usage'],
        'temperature': temperature,
        'prompt': prompt,
        "assistant_reply": response['choices'][0]['message']['content']
    }
    
    next_index = len(list(path_to_dataset.glob('*')))
    content = response['choices'][0]['message']['content']
    output_filename = f'{next_index}_{len(content)}_{str(temperature)}.json'

    try:
        with open(path_to_dataset / output_filename, 'w') as json_file:
            json.dump(response_metadata, json_file, indent=4, ensure_ascii=False)
            
    except (UnicodeDecodeError, UnicodeEncodeError, JSONDecodeError):
        with open(path_to_dataset / output_filename, 'w') as json_file:
            json.dump(response_metadata, json_file, indent=4)

 95%|█████████▌| 190/200 [2:08:29<09:52, 59.27s/it]  

In [5]:
sum_used_tokens = 0
character_count = 0

for file_path in path_to_dataset.glob('*'):
    try:
        with open(file_path, "r") as json_file:
            json_data = json.load(json_file)
            sum_used_tokens += json_data['usage']['total_tokens']
            character_count += len(json_data['assistant_reply'])
            
    except JSONDecodeError:
        file_path.unlink()
        print(f'Deleted {file_path}')

print(f'Sum of used tokens: {sum_used_tokens}\nCharacter count: {character_count}')

Deleted dataset\644_2484_0.8.json
Sum of used tokens: 2536123
Character count: 6145146


In [44]:
# for filepath in path_to_metadata.iterdir():
#     if filepath.suffix == '.json':  # Change this condition to match your file type
#         if str(filepath)[-8:] == '0.9.json':
# 
#             parts = str(filepath.name).split('_')
#             joined_name = f'{parts[0]}_{parts[-2]}_{parts[-1]}'
#             
#             new_filename = filepath.name.replace(filepath.name, joined_name)  
#             new_path = filepath.with_name(new_filename)
#             
#             filepath.rename(new_path)
#             print(f'Renamed {filepath.name} to {new_filename}')
#             
#         else:
#             
#             parts = str(filepath.name).split('_')
#             joined_name = f'{parts[0]}_{parts[-1][:-5]}_0.8.json'
#             
#             new_filename = filepath.name.replace(filepath.name, joined_name)  
#             new_path = filepath.with_name(new_filename)
#             
#             filepath.rename(new_path)
#             print(f'Renamed {filepath.name} to {new_filename}')
#             
#             
#             
# for filepath in path_to_raw_data.iterdir():
#     if filepath.suffix == '.json':  # Change this condition to match your file type
#         if str(filepath)[-8:] == '0.9.json':
# 
#             parts = str(filepath.name).split('_')
#             joined_name = f'{parts[0]}_{parts[-2]}_{parts[-1]}'
#             
#             new_filename = filepath.name.replace(filepath.name, joined_name)  
#             new_path = filepath.with_name(new_filename)
#             
#             filepath.rename(new_path)
#             print(f'Renamed {filepath.name} to {new_filename}')
#             
#         else:
#             
#             parts = str(filepath.name).split('_')
#             joined_name = f'{parts[0]}_{parts[-1][:-5]}_0.8.json'
#             
#             new_filename = filepath.name.replace(filepath.name, joined_name)  
#             new_path = filepath.with_name(new_filename)
#             
#             filepath.rename(new_path)
#             print(f'Renamed {filepath.name} to {new_filename}')